# Hotel Booking Cancellation Analysis with Revenue Impact

## 1. Business Problem

Hotel booking cancellations create significant revenue uncertainty and operational challenges. 
Understanding when cancellations occur, which customer segments cancel more frequently,
and how much revenue is at risk is critical for improving pricing, overbooking, and customer management strategies.


## 2. Dataset Overview

The dataset contains hotel booking records including booking details, customer characteristics, stay information, and cancellation status.  
The target variable for analysis is `is_canceled`, which indicates whether a booking was canceled.  
This dataset is used to analyze cancellation patterns and quantify potential revenue loss.

In [30]:
import pandas as pd

df = pd.read_csv(r"C:\Users\Krishna\data projects\hotel_booking_folder\hotel_booking.csv")
df.head()


,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,name,email,phone-number,credit_card
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,Transient,0.0,0,0,Check-Out,2015-07-01,Ernest Barnes,Ernest.Barnes31@outlook.com,669-792-1661,************4322
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,Transient,0.0,0,0,Check-Out,2015-07-01,Andrea Baker,Andrea_Baker94@aol.com,858-637-6955,************9157
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,Transient,75.0,0,0,Check-Out,2015-07-02,Rebecca Parker,Rebecca_Parker@comcast.net,652-885-2745,************3734
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,Transient,75.0,0,0,Check-Out,2015-07-02,Laura Murray,Laura_M@gmail.com,364-656-8427,************5677
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,Transient,98.0,0,1,Check-Out,2015-07-03,Linda Hines,LHines@verizon.com,713-226-5883,************5498


In [31]:
df.shape

(119390, 36)

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 36 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           119390 non-null  object 
 1   is_canceled                     119390 non-null  int64  
 2   lead_time                       119390 non-null  int64  
 3   arrival_date_year               119390 non-null  int64  
 4   arrival_date_month              119390 non-null  object 
 5   arrival_date_week_number        119390 non-null  int64  
 6   arrival_date_day_of_month       119390 non-null  int64  
 7   stays_in_weekend_nights         119390 non-null  int64  
 8   stays_in_week_nights            119390 non-null  int64  
 9   adults                          119390 non-null  int64  
 10  children                        119386 non-null  float64
 11  babies                          119390 non-null  int64  
 12  meal            

## 3. Data Cleaning 

The dataset was cleaned to ensure reliable analysis.  
Invalid records such as negative average daily rates (ADR) and bookings with zero adults were removed.  
Missing values were handled using business context to preserve meaningful information.


In [33]:
# Check missing values in each column
df.isna().sum().sort_values(ascending=False)


company                           112593
agent                              16340
country                              488
children                               4
arrival_date_year                      0
lead_time                              0
is_canceled                            0
hotel                                  0
stays_in_weekend_nights                0
stays_in_week_nights                   0
arrival_date_day_of_month              0
arrival_date_month                     0
babies                                 0
meal                                   0
market_segment                         0
distribution_channel                   0
is_repeated_guest                      0
previous_cancellations                 0
adults                                 0
arrival_date_week_number               0
reserved_room_type                     0
previous_bookings_not_canceled         0
booking_changes                        0
assigned_room_type                     0
deposit_type    

In [34]:
# Summary statistics including categorical columns
df.describe(include="all").T


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
hotel,119390,2,City Hotel,79330,NaN,NaN,NaN,NaN,NaN,NaN,NaN
is_canceled,119390.0,NaN,NaN,NaN,0.370416,0.482918,0.0,0.0,0.0,1.0,1.0
lead_time,119390.0,NaN,NaN,NaN,104.011416,106.863097,0.0,18.0,69.0,160.0,737.0
arrival_date_year,119390.0,NaN,NaN,NaN,2016.156554,0.707476,2015.0,2016.0,2016.0,2017.0,2017.0
arrival_date_month,119390,12,August,13877,NaN,NaN,NaN,NaN,NaN,NaN,NaN
arrival_date_week_number,119390.0,NaN,NaN,NaN,27.165173,13.605138,1.0,16.0,28.0,38.0,53.0
arrival_date_day_of_month,119390.0,NaN,NaN,NaN,15.798241,8.780829,1.0,8.0,16.0,23.0,31.0
stays_in_weekend_nights,119390.0,NaN,NaN,NaN,0.927599,0.998613,0.0,0.0,1.0,2.0,19.0
stays_in_week_nights,119390.0,NaN,NaN,NaN,2.500302,1.908286,0.0,1.0,2.0,3.0,50.0
adults,119390.0,NaN,NaN,NaN,1.856403,0.579261,0.0,2.0,2.0,2.0,55.0


In [35]:
# Check for impossible or suspicious numeric values
df[['adults', 'children', 'babies', 'adr', 'lead_time']].describe().T


,count,mean,std,min,25%,50%,75%,max
adults,119390.0,1.856403,0.579261,0.00,2.00,2.000,2.0,55.0
children,119386.0,0.103890,0.398561,0.00,0.00,0.000,0.0,10.0
babies,119390.0,0.007949,0.097436,0.00,0.00,0.000,0.0,10.0
adr,119390.0,101.831122,50.535790,-6.38,69.29,94.575,126.0,5400.0
lead_time,119390.0,104.011416,106.863097,0.00,18.00,69.000,160.0,737.0


In [36]:
 # count problematic rows
print("ADR < 0:", (df['adr'] < 0).sum())
print("Adults = 0:", (df['adults'] == 0).sum())


ADR < 0: 1
Adults = 0: 403


In [37]:
# remove rows with negative ADR or zero adults
df = df[(df['adr'] >= 0) & (df['adults'] > 0)]


In [38]:
#cross checking 
print("ADR < 0:", (df['adr'] < 0).sum())
print("Adults = 0:", (df['adults'] == 0).sum())

ADR < 0: 0
Adults = 0: 0


“I identified and removed invalid records such as negative room rates and bookings with zero adults to ensure data quality before analysis.”

In [39]:
# make an explicit copy to avoid chained assignment warnings
df = df.copy()


In [40]:
# fill missing children values with 0
df['children'] = df['children'].fillna(0)


In [41]:
df['children'].isna().sum()


np.int64(0)

In [42]:
#filling missing country values
df['country'] = df['country'].fillna(0)
df['country'].isna().sum()

np.int64(0)

In [43]:
#filling missing agent values
df['agent'] = df['agent'].fillna(0)
df['agent'].isna().sum()

np.int64(0)

In [44]:
# fill missing company values
df['company'] = df['company'].fillna(0)
df['company'].isna().sum()


np.int64(0)

In [46]:
#ALL missing values are handled
df.isna().sum().sum()

np.int64(0)

## 4. Feature Engineering

creating useful columns(columns that help answer business questions)

Additional features were created to support business-focused analysis, including total stay duration, 
booking value as a revenue proxy,and lead time categories to assess cancellation risk.


In [48]:
# total number of nights stayed
df['total_stay_nights'] = (
    df['stays_in_weekend_nights'] + df['stays_in_week_nights']
)


In [49]:
df[['stays_in_weekend_nights',
    'stays_in_week_nights',
    'total_stay_nights']].head()


,stays_in_weekend_nights,stays_in_week_nights,total_stay_nights
0,0,0,0
1,0,0,0
2,0,1,1
3,0,1,1
4,0,2,2


In [50]:
# booking value (revenue proxy)
df['booking_value'] = df['adr'] * df['total_stay_nights']


In [51]:
df[['adr', 'total_stay_nights', 'booking_value']].head()


,adr,total_stay_nights,booking_value
0,0.0,0,0.0
1,0.0,0,0.0
2,75.0,1,75.0
3,75.0,1,75.0
4,98.0,2,196.0


##### Lead time buckets

coz cancellation risk changes based on how early someone books

In [52]:
# create lead time buckets
def lead_time_bucket(days):
    if days <= 7:
        return '0-7 days'
    elif days <= 30:
        return '8-30 days'
    elif days <= 90:
        return '31-90 days'
    else:
        return '90+ days'

df['lead_time_bucket'] = df['lead_time'].apply(lead_time_bucket)


In [53]:
df['lead_time_bucket'].value_counts()


lead_time_bucket
90+ days      50976
31-90 days    29475
0-7 days      19617
8-30 days     18918
Name: count, dtype: int64

## 5. Cancellation & Revenue Analysis ⏳

In [54]:
# Business Questions:
# 1. Who cancels more?
# 2. When do customers cancel?
# 3. How much revenue is at risk due to cancellations?

In [56]:
#  Overall cancellation rate 
df['is_canceled'].mean()

np.float64(0.37075790429126115)

“The overall cancellation rate was ~37%, indicating significant revenue risk.”

In [57]:
# When do customers cancel?
# Cancellation rate by lead time bucket
cancel_by_lead_time = (
    df.groupby('lead_time_bucket')['is_canceled']
      .mean()
      .reset_index()
      .sort_values('is_canceled', ascending=False)
)

cancel_by_lead_time


,lead_time_bucket,is_canceled
3,90+ days,0.506807
1,31-90 days,0.377439
2,8-30 days,0.278676
0,0-7 days,0.095988


This is the average cancellation rate for each booking lead-time group
(i.e., probability that a booking gets canceled based on how early it was made).
Bookings made more than 90 days in advance had the highest cancellation rate (~51%), 
while last-minute bookings (0–7 days) were the most reliable (~10%).

In [58]:
# Cancellation rate by customer type(who cancels more)
cancel_by_customer = (
    df.groupby('customer_type')['is_canceled']
      .mean()
      .reset_index()
      .sort_values('is_canceled', ascending=False)
)

cancel_by_customer


,customer_type,is_canceled
2,Transient,0.407827
0,Contract,0.309998
3,Transient-Party,0.254389
1,Group,0.101222


Transient customers showed the highest cancellation rate (~41%), while group bookings were the most reliable (~10%),
indicating individual bookings pose higher revenue risk.

In [60]:
# Revenue at risk due to cancellations
revenue_at_risk = round(
    df[df['is_canceled'] == 1]['booking_value']
      .sum(),2
)

revenue_at_risk


np.float64(16699302.07)

👉 ~16.7 million (hotel currency units) worth of bookings were canceled.

## 6. Key Insights & Business Recommendations



### Key Insights
- Bookings made more than 90 days in advance showed the highest cancellation rates(~50%).
- Transient customers had significantly higher cancellation(~40%) rates compared to group bookings(~10%).
- Approximately 16.7 million in booking value was at risk due to canceled reservations.

### Business Recommendations
- Apply stricter cancellation policies or deposits for long lead-time bookings.
- Use controlled overbooking strategies for high-risk customer segments.
- Prioritize and incentivize reliable segments such as group bookings.
